In [ ]:
#clean the data

xls = pd.ExcelFile('The_data.xls')
df_age_sex = pd.read_excel(xls, 'Age and Sex')
df_elevation = pd.read_excel(xls, 'Elevation')
df_marital_status = pd.read_excel(xls, 'Marital Status')
df_educational_attainment = pd.read_excel(xls, 'Educational Attainment')
df_housing_units_type = pd.read_excel(xls, 'Housing Units Type')
df_housing_value = pd.read_excel(xls, 'Housing Value')
df_year_house_built = pd.read_excel(xls, 'Year Structure Built')
df_v = pd.read_excel(xls, 'Age and Sex')


